# Part1: How to Ensure Your Changes Are BIDS Compliant

In the event that you changed BIDS data manually, we want to ensure that the changes are BIDS compliant, because the flywheel gear that checks this for us automatically, only does so at upload time. So, below we demonstrate the use of a module/script `validate_bids_changes.py` that checks the difference between an original BIDS query csv and a one that has been modified by the user.

In [1]:
import flywheel
import pandas
# add the script to the path
import sys
import os
sys.path.append(os.path.abspath("/home/ttapera/bids-on-flywheel/scripts"))
import upload_bids as upload

The module has a few important functions:

1. `read_flywheel_csv(f_path)`

This is a wrapper around pandas' `read_csv`, but ensures that the columns match a flywheel query

2. `get_unequal_cells(original_df, modified_df)`

This function will return indices of cells that are not exact matches between the original and modified csvs

3. `validate_unequal_cells(unequal_indices, modified_df)`

This function goes through each of the unequal cell pairs and makes sure that the changes applied are valid for the column type; so you can't put a number in a string column, or (God-forbid) missname your BIDS session label!

The erroneous changes will print to stdout.

In [2]:
df = upload.read_flywheel_csv("./data/reward_audit.csv")

Now, let's make a copy of this dataframe and modify it:

In [3]:
df_modified = df.copy()

df_modified.loc[1, 'acquisition.label'] = "a new scan"  # valid string field
df_modified.loc[1, 'valid'] = "TRUE"  # valid boolean field
df_modified.loc[1, 'Modality'] = 'MR'  # valid drop down choice field
df_modified.loc[4, 'Task'] = 89  # invalid string field
df_modified.loc[4, 'valid'] = 'T'  # invalid boolean field
df_modified.loc[4, 'Modality'] = 'magnetic resonance image'  # invalid drop down choice field
df_modified.loc[4, 'session.label'] = "some new name" # invalid bids session name
df_modified.loc[4, 'acquisition.id'] = "x" # NEVER change this, it's necessary to find the object in flywheel

You can see that the two dataframes are different in lines 2 and 5 (note that Python uses zero-indexing, so lines in the dataframes below start at zero):

In [4]:
df.head()

,acquisition.label,valid,acquisition.id,project.label,session.label,subject.label,Filename,Folder,IntendedFor,Mod,Modality,Path,Rec,Run,Task,error_message,ignore,template
0,bbl1_restbold_mb6_742,False,5c1a70239011bd001436894e,Reward2018,7944,100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ep2d_itc1_168,False,5c1a70239011bd001536887c,Reward2018,7944,100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bbl1_cardB0_178,False,5c1a70239011bd001536887d,Reward2018,7944,100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b0map_v4,False,5c1a70239011bd00113688a2,Reward2018,7944,100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bbl1_cardA0_178,False,5c1a70239011bd001436894f,Reward2018,7944,100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_modified.head()

,acquisition.label,valid,acquisition.id,project.label,session.label,subject.label,Filename,Folder,IntendedFor,Mod,Modality,Path,Rec,Run,Task,error_message,ignore,template
0,bbl1_restbold_mb6_742,False,5c1a70239011bd001436894e,Reward2018,7944,100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a new scan,TRUE,5c1a70239011bd001536887c,Reward2018,7944,100088,NaN,NaN,NaN,NaN,MR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bbl1_cardB0_178,False,5c1a70239011bd001536887d,Reward2018,7944,100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b0map_v4,False,5c1a70239011bd00113688a2,Reward2018,7944,100088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bbl1_cardA0_178,T,x,Reward2018,some new name,100088,NaN,NaN,NaN,NaN,magnetic resonance image,NaN,NaN,NaN,89,NaN,NaN,NaN


And now, we compare the two dataframes, and validate each change:

In [6]:
# check for equality of each cell between the original and modified
unequal = upload.get_unequal_cells(df, df_modified)
# if any unequal, assess the validity of the modification
result = upload.validate_on_unequal_cells(unequal, df_modified)

The following changes don't seem to be valid for this data:

Row 5, Column 2, "T"
This field accepts booleans, these can only be written as "True" or "False"!

Row 5, Column 3, "x"
You cannot edit the acquisition ID!

Row 5, Column 5, "some new name"
This field MUST be a BIDS compliant name!

Row 5, Column 11, "magnetic resonance image"
This field must match one of the available options in the drop-down menu on the website!

Row 5, Column 15, "89"
This field only accepts strings!


For the sake of communicability we print the messages with indexing starting at 1.

Note that `get_unequal_cells()` has a default argument `provenance=True` that write out a text log to the working directory, comprising of the changes that were made between the original and modified files.


# Part 2: Uploading Your BIDS Changes


Just like before, you can upload changes to flywheel by using the flywheel SDK and the `acquisition.update_info()` function. Let's create another modification that this time is valid for the participant we had edited earlier:

In [7]:
df_modified = df.copy()

df_modified[df_modified['subject.label'] == 19830]

,acquisition.label,valid,acquisition.id,project.label,session.label,subject.label,Filename,Folder,IntendedFor,Mod,Modality,Path,Rec,Run,Task,error_message,ignore,template
2362,ep2d_effort3_1416,False,5c1a82ca9011bd0013369086,Reward2018,neff2,19830,sub-19830_ses-neff2_task-{file.info.BIDS.Task}...,func,NaN,NaN,bold,sub-19830/ses-neff2/func,NaN,NaN,NaN,Task u'' does not match '^[a-zA-Z0-9]+$',False,func_file
2363,ep2d_effort1_236,False,5c1a82ca9011bd0011368ecf,Reward2018,neff2,19830,sub-19830_ses-neff2_task-{file.info.BIDS.Task}...,func,NaN,NaN,sbref,sub-19830/ses-neff2/func,NaN,NaN,NaN,Task u'' does not match '^[a-zA-Z0-9]+$',False,func_file
2364,MPRAGE_TI1100_ipat2,True,5c1a82ca9011bd0013369087,Reward2018,neff2,19830,sub-19830_ses-neff2_T1w.nii.gz,anat,NaN,NaN,T1w,sub-19830/ses-neff2/anat,NaN,NaN,NaN,NaN,False,anat_file
2365,sag mpr,False,5c1a82ca9011bd00143693a7,Reward2018,neff2,19830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2366,ep2d_effort2_236,False,5c1a82ca9011bd0011368ed0,Reward2018,neff2,19830,sub-19830_ses-neff2_task-{file.info.BIDS.Task}...,func,NaN,NaN,sbref,sub-19830/ses-neff2/func,NaN,NaN,NaN,Task u'' does not match '^[a-zA-Z0-9]+$',False,func_file
2367,foo,False,5c1a82ca9011bd0011368ed1,Reward2018,neff2,19830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2368,PhoenixZIPReport,False,5c1a82ca9011bd00143693a8,Reward2018,neff2,19830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2369,B0map_onesizefitsall_v4,False,5c1a82ca9011bd0013369088,Reward2018,neff2,19830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2370,localizer,False,5c1a82ca9011bd0013369089,Reward2018,neff2,19830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2371,ep2d_single,False,5c1a82ca9011bd00143693a9,Reward2018,neff2,19830,sub-19830_ses-neff2_task-{file.info.BIDS.Task}...,func,NaN,NaN,sbref,sub-19830/ses-neff2/func,NaN,NaN,NaN,Task u'' does not match '^[a-zA-Z0-9]+$',False,func_file


In [8]:
df_modified.loc[2366, 'Task'] = "Effort2"
df_modified.loc[2366, 'valid'] = "True"
df_modified.loc[2366, 'Filename'] = "sub-19830_ses-neff2_task-effort_run-2_sbref.nii.gz"

In [9]:
# check for equality of each cell between the original and modified
unequal = upload.get_unequal_cells(df, df_modified)
# if any unequal, assess the validity of the modification
is_valid = upload.validate_on_unequal_cells(unequal, df_modified)

# were the changes valid?
is_valid

True

Great, now we use the upload function in validate, which grabs the acquisition ID, and the column you modified, and uploads!

In [10]:
fw = flywheel.Client()

In [11]:
upload.upload_to_flywheel(change_index=unequal, client=fw, modified_df=df_modified)

Adding change 1 of 3
Adding change 2 of 3
Adding change 3 of 3


You can see the changes have been applied here:

<img src="../images/error_removed2.png" alt="drawing" width="600"/>

This process can be run in the shell as a one liner:

`python upload_bids.py original.csv modified.csv`